In [1]:
import numpy as np
import os
import glob
import sys
import torch
from pathlib import Path

current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
grandparent_dir = os.path.abspath(os.path.join(parent_dir, '..'))
sys.path.append(grandparent_dir)
print(grandparent_dir)

import gendata
import learner as ln
from utils import Logger, read_config_file, set_random_seed, tensors_to_numpy

from learner.metric.dynamics_metric import (
    calculate_dynamics_metrics,
    plot_dynamics_metrics,
)

/home/lbu/project/PINN_DE


In [2]:
from configs.config_plot import *

os.makedirs(output_dir, exist_ok=True)  

seed = 0
set_random_seed(seed)

# Logger
logger = Logger(output_dir)

config_file_path = "/home/lbu/project/PINN_DE/configs/test_something/config_model_test.py"
config = read_config_file(config_file_path)

logger.info("#" * 100)

2023-08-27 19:54:43 INFO ####################################################################################################


In [17]:
########################################################
#
# data setting
#
########################################################

physics_t = torch.linspace(0, 3, 20, device=config.device, dtype=config.dtype).view(-1, 1)
data_t = torch.linspace(0, 3, 10, device=config.device, dtype=config.dtype).view(-1, 1)

y0 = torch.tensor([0., 1.], device=config.device, dtype=config.dtype)
y = torch.cat([torch.sin(data_t), torch.cos(data_t)], dim=-1)
yt = torch.cat([torch.cos(data_t), -torch.sin(data_t)], dim=-1)

physics_t = physics_t.requires_grad_(True)
data_t = data_t.requires_grad_(True)

data = y0, y, yt, data_t, physics_t

# system

In [4]:
from torch import nn, Tensor

class SinActivation(nn.Module):

    def forward(self, input: Tensor) -> Tensor:
        return torch.sin(input)


class MLPBlock(nn.Module):
    """
    MLPBlock represents a single block of the MLP (Multi-Layer Perceptron) model.

    It consists of a linear layer followed by an activation function.

    Args:
        input_dim (int): The number of input features to the block.
        output_dim (int): The number of output features from the block.
        activation (torch.nn.Module): The activation function to be applied after the linear layer.
    """

    def __init__(self, input_dim, output_dim, activation):
        super(MLPBlock, self).__init__()
        self.linear_layer = nn.Linear(input_dim, output_dim)
        self.activation = activation

    def forward(self, x):
        """
        Forward pass of the MLPBlock.

        Args:
            x (torch.Tensor): The input tensor of shape (batch_size, input_dim).

        Returns:
            torch.Tensor: The output tensor of shape (batch_size, output_dim).
        """
        return self.activation(self.linear_layer(x))

In [26]:
# encoding: utf-8

import os
from matplotlib import pyplot as plt

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

from learner.metric.dynamics_metric import (
    calculate_dynamics_metrics,
    plot_dynamics_metrics,
)
from utils import batched_jacobian, initialize_class, tensors_to_numpy


class BackboneNet(nn.Module):
    """
    Custom backbone neural network module.

    This module contains multiple MLPBlocks with a specified number of layers.

    Args:
        input_dim (int): Dimension of the input features.
        hidden_dim (int): Dimension of the hidden layers.
        output_dim (int): Dimension of the output.
        layers_num (int): Number of hidden layers in the network.
    """

    def __init__(self, config):
        super(BackboneNet, self).__init__()

        input_dim = config.BackboneNet_input_dim
        hidden_dim = config.BackboneNet_hidden_dim
        output_dim = config.BackboneNet_output_dim
        layers_num = config.BackboneNet_layers_num

        activation = SinActivation()

        # Input layer
        input_layer = MLPBlock(input_dim, hidden_dim, activation)

        # Hidden layers
        hidden_layers = nn.ModuleList(
            [MLPBlock(hidden_dim, hidden_dim, activation) for _ in range(layers_num)]
        )

        # Output layer
        output_layer = nn.Linear(hidden_dim, output_dim, bias=False)

        layers = []
        layers.extend([input_layer])
        layers.extend(hidden_layers)
        layers.extend([output_layer])

        # Create the sequential model
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        """
        Forward pass of the BackboneNet.

        Args:
            x (torch.Tensor): The input tensor of shape (batch_size, input_dim).

        Returns:
            torch.Tensor: The output tensor of shape (batch_size, output_dim).
        """
        out = self.net(x)
        return out


class PINN(nn.Module):
    def __init__(self, config, logger, *args, **kwargs):
        super(PINN, self).__init__()

        self.config = config
        self.logger = logger

        self.device = config.device
        self.dtype = config.dtype

        self.backboneNet = BackboneNet(config)

        try:
            class_name = self.config.dynamic_class
            kwargs = {"config": config, "logger": logger}
            self.right_term_net = initialize_class("dynamics", class_name, **kwargs)
        except ValueError as e:
            raise RuntimeError("class '{}' is not available".format(class_name))

    def forward(self, t, q0):
        bs, _ = t.shape
        t = t.reshape(-1, 1)
        out = torch.zeros(bs, 2, device=self.device, dtype=self.dtype)
        qi =  q0
        for i, ti in enumerate(t):
            input = torch.cat([ti,qi], dim=-1)
            qi = self.backboneNet(input)
            out[i] = qi
        return out

    def get_q_qt_qtt(self, t, q0):
        q_hat = self(t, q0)
        qt_hat = batched_jacobian(
            q_hat, t, device=self.device, dtype=self.dtype
        ).squeeze(-1)
        qtt_hat = batched_jacobian(
            qt_hat, t, device=self.device, dtype=self.dtype
        ).squeeze(-1)
        return q_hat, qt_hat, qtt_hat


config.BackboneNet_input_dim = 3
config.BackboneNet_hidden_dim = 20
config.BackboneNet_output_dim = 2
config.BackboneNet_layers_num = 3
net_dp = PINN(config, logger).to(config.device).to(config.dtype)
net_dp

PINN(
  (backboneNet): BackboneNet(
    (net): Sequential(
      (0): MLPBlock(
        (linear_layer): Linear(in_features=3, out_features=20, bias=True)
        (activation): SinActivation()
      )
      (1): MLPBlock(
        (linear_layer): Linear(in_features=20, out_features=20, bias=True)
        (activation): SinActivation()
      )
      (2): MLPBlock(
        (linear_layer): Linear(in_features=20, out_features=20, bias=True)
        (activation): SinActivation()
      )
      (3): MLPBlock(
        (linear_layer): Linear(in_features=20, out_features=20, bias=True)
        (activation): SinActivation()
      )
      (4): Linear(in_features=20, out_features=2, bias=False)
    )
  )
  (right_term_net): DynamicDoublePendulumDAE(
    (phi_net): Phi_Net()
    (m_net): M_Net()
    (f_net): F_Net()
  )
)

In [34]:
q_hat = net_dp(data_t, y0)
q_hat.shape

q_hat, qt_hat, qtt_hat = net_dp.get_q_qt_qtt(data_t, y0)
qtt_hat

tensor([[-0.0141, -0.0017],
        [-0.0056,  0.0022],
        [-0.0037,  0.0050],
        [-0.0025,  0.0066],
        [-0.0017,  0.0074],
        [-0.0013,  0.0073],
        [-0.0013,  0.0067],
        [-0.0015,  0.0059],
        [-0.0017,  0.0050],
        [-0.0022,  0.0039]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SqueezeBackward1>)

In [18]:
y0

tensor([0., 1.], device='cuda:0', dtype=torch.float64)